In [3]:
import os
import pandas as pd
from glob import glob

In [ ]:
proj_dict = {
    'TB_fire_clip.csv': {
        'Project': 'Fire',
        'Method': 'clip',
        'ID': 'Transect ID',
        'Date': 'Date',
        'bm_gm2': 'Total_Weight_(g/m^2)',
        'gps_E': 'Mean.GPS_E.Biomass',
        'gps_N': 'Mean.GPS_N.Biomass'
    },
    'TB_nex_clip.csv': {
        'Project': 'NEX',
        'Method': 'clip',
        'ID': ['Site', 'Disturbance', 'Exclosure']
        'Date': 'Date_cln',
        'bm_gm2': 'Total Weight (g/m^2)',
        'gps_E': 
    }
}

In [8]:
clipLIST = glob('../data/clip/*.csv')

In [9]:
clipLIST

['../data/clip/TB_fire_clip.csv',
 '../data/clip/TB_nex_clip.csv',
 '../data/clip/TB_pdog_clip.csv',
 '../data/clip/TB_wtgn_clip.csv']

In [20]:
test = pd.read_csv(clipLIST[1])

In [26]:
test.groupby(['Disturbance', 'Site', 'Exclosure', 'Date_cln']).Rep.unique()#.columns

Disturbance  Site        Exclosure     Date_cln  
control      lake creek  livestock ex  2015-07-27    [2.0]
                                       2016-08-02    [2.0]
                                       2017-07-25    [2.0]
                                       2018-06-07    [2.0]
                                       2018-07-03    [2.0]
                                                     ...  
prairie dog  wildcat     ungulate ex   2019-07-03    [2.0]
                                       2019-08-05    [2.0]
                                       2020-07-13    [2.0]
                                       2020-08-06    [1.0]
                                       2021-07-12    [2.0]
Name: Rep, Length: 385, dtype: object

In [31]:
test.columns

Index(['Site', 'Disturbance', 'Exclosure', 'Date_cln', 'Year', 'Rep',
       'Area (m^2)', '% sd ARTR', '% ARCA', '% ATCA', '# OPPO cladodes',
       '# COVI', '% OPPO', 'Total Weight (g)', 'Total Weight (g/m^2)',
       'C4 weight (g)', 'C3 weight (g)', 'Bromes weight (g)',
       'Forbs weight (g)', 'Ann Forb weight (g)', 'Per Forb weight (g)',
       'VUOC weight (g)', 'Subshrub weight (g)', 'Standing Dead weight (g)',
       'Check'],
      dtype='object')